# Spellchecker quest

Хтось наробив помилок у віршах Тараса Шевченка. Наша задача -- виправити ці помилки і прочитати приховане повідомлення.

## Задача

Ви отримаєте тренувальні та тестувальні дані.

Тренувальні дані знаходяться в полі `lab.train_text`. Це звичайний нерозмічений текст. На ньому необхідно натренувати мовну модель. Підійде будь-яка. Я би радив feed-forward нейрону модель з токенізацією по літерах, бо це те, що ми проходили на останній лекції. Але n-грамна теж має спрацювати.

Тестувальні дані знаходиться в полі `lab.test_items`. Приклад одного елемента:

```json
{
  "text": "Співали б прозу, та по ножах,",
  "error_start": 23,
  "error_end": 28,
  "error": "ножах",
  "corrections": [
    "ногах",
    "йотах",
    "єнотах",
    "ножах",
    "нотах"
  ]
}
```

`error_start` та `error_end` вказують на місцезнаходження помилки в тексті (в символах). У данному прикладі, помилкою є `text[23:28]`, тобто слово "ножах".

`corrections` -- це список можливих виправлень.

Ваша задача -- обрати правильне виправлення серед запропонованих.


## Приховане повідомлення

Один приклад в `lab.test_items` дає можливість прочитати одну літеру прихованого повідомлення. Для цього знайдіть різницю між літерами слова з помилкою (`error`) та обраним виправленням. Надрукуйте цю літеру. Якщо слово з помилкою направді правильне, а таке теж буває, надрукуйте пробіл. Приклади:

```
Error               Correction     To print
-------------------------------------------
привіт               приліт        л
пні                  поні          о
баллет               балет         л
привіт               привіт        (space)
```

Приховане повідомлення, яке ви побачите в результаті це рядок з віршу одного з українських авторів.

Відповідь на квест -- ім'я автора/ки у форматі "Ім'я Прізвище".

Полетіли! 🚀

In [1]:
!pip install --quiet --ignore-installed http://nlp.band/static/pypy/lpnlp-2023.10.2-py3-none-any.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.6 MB/s eta 0:00:00


In [3]:
import lpnlp

lab = lpnlp.start(
    email="vasyl.rusyn.kn.2021@lpnu.ua",                   # <----------- Заповніть це поле
    lab="quest_spellchecker"
    )

Удачі!


## Мовна модель

Натренуйте свою мовну модель тут

#Imports

In [14]:
from tqdm import tqdm
import torch
from torch import nn
from typing import Iterable

#Classes

In [9]:
class Vocabulary:

  def __init__(self, tokens, unk_token="<unk>"):
    self.unk_token = unk_token
    self.unk_index = 0
    self._itos = set([unk_token] + tokens)
    self._stoi = {token: index for index, token in enumerate(self._itos)}

  def stoi(self, token: str) -> int:
    """Return token index or `<unk>` index if `token` is not in the vocab.
    """
    return self._stoi.get(token, self.unk_index)


  def itos(self, index: int) -> str:
    """Return token by its `index`.

    Raise LookupError if `index` is out of vocabulary range.
    """

    return self._itos[index]

  @property
  def tokens(self):
    return self._itos

  def __len__(self) -> int:
    return len(self._itos)

class BengioLMModel(nn.Module):
    def __init__(self, vocab_size: int, embed_dim: int, context_len: int, hidden_dim: int) -> None:

        super().__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.context_len = context_len
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(vocab_size, embed_dim) # vocab_size * embed_dim
        self.W = nn.Linear(context_len * embed_dim, hidden_dim)
        self.tanh = nn.Tanh()
        self.U = nn.Linear(hidden_dim, vocab_size)

    def forward(self, X_indexes: torch.tensor):

        """

        Args:
            X_indexes: tensor of indexes of context tokens.
        """

        X = self.embed(X_indexes) # [batch_size, context len * embed dim]
        e = X.view(-1, self.context_len * self.embed_dim)

        h = self.tanh(self.W(e))

        logits = self.U(h)

        log_probs = torch.log_softmax(logits, dim=-1)

        return log_probs

#Helping functions

In [15]:
def tokenize(text: str) -> [str]:
   return list(text.lower())

def prepare_data(tokens: [str], context_len: int) -> [([str], str)]:
    """

    Args:
        tokens: list of tokens
        context_len: length of context

    Reurns:
        Iterable of (context, target) pairs
    """
    # res = []

    for i in range(context_len, len(tokens)):
        context = tokens[i - context_len:i]
        target = tokens[i]

        yield (context, target)

def batch_it(xs, batch_size):
    batch = []

    for i, x in enumerate(xs):
        batch.append(x)

        if i % batch_size == batch_size - 1:
            yield batch
            batch = []

    if batch:
        yield batch

def vectorize(tokens: Iterable[str], vocab: Vocabulary) -> torch.tensor:
   X = torch.tensor([vocab.stoi(token) for token in tokens])
   return X

In [4]:
print(lab.train_text[:330])

﻿ПРИЧИННА

Реве та стогне Дніпр широкий,
Сердитий вітер завива,
Додолу верби гне високі,
Горами хвилю підійма.
І блідий місяць на ту пору
Із хмари де-де виглядав,
Неначе човен в синім морі,
То виринав, то потопав.
Ще треті півні не співали,
Ніхто нігде не гомонів,
Сичі в гаю перекликались,
Та ясен раз у раз скрипів.


In [28]:
############################################

##### Ваш код для тренування моделі тут

train_text_tokens = tokenize(lab.train_text)
vocab = Vocabulary(train_text_tokens)
print(len(vocab))

hparams = {
        "vocab_size": len(vocab),
        "embed_dim": 128,
        "context_len": 15,
        "hidden_dim": 256,
        "learning_rate": 0.01,
        "num_epochs": 10,
        "batch_size": 4096
    }

model = BengioLMModel(vocab_size=hparams["vocab_size"], embed_dim=hparams["embed_dim"],
                          context_len=hparams["context_len"], hidden_dim=hparams["hidden_dim"])

optimizer = torch.optim.Adam(model.parameters(), lr=hparams["learning_rate"])
loss_fn = nn.NLLLoss()

best_loss = 999999
eraly_stop = 0

for epoch in range(hparams["num_epochs"]):

  total_loss = 0.0
  examples = prepare_data(train_text_tokens, hparams["context_len"])
  examples = list(examples)

  for batch in tqdm(batch_it(examples, batch_size=hparams["batch_size"]), leave=False):

    X_batch = []
    y_batch = []

    for context, target in batch:
      X = vectorize(context, vocab)
      y = vectorize([target], vocab)
      X_batch.append(X)
      y_batch.append(y)

    X_batch = torch.stack(X_batch)
    y_batch = torch.tensor(y_batch)

    optimizer.zero_grad()

    log_probs = model(X_batch)
    loss = loss_fn(log_probs, y_batch)

    loss.backward()
    optimizer.step()
    total_loss += loss.sum().item()

  print(f" Epoch: {epoch} Loss: {total_loss / len(examples)}")

  if total_loss / len(examples) < best_loss:
    best_loss = total_loss / len(examples)
    early_stop = 0

  else:
    eraly_stop += 1

  if early_stop == 3:
    break




# vocab = ...
# model = ...

# ...

############################################

82


 Epoch: 0 Loss: 0.0006280367939708394


 Epoch: 1 Loss: 0.0005592429849966121


 Epoch: 2 Loss: 0.0005381619467563101


 Epoch: 3 Loss: 0.0005244605601712597


...

 Epoch: 5 Loss: 0.0005046109520243945


 Epoch: 6 Loss: 0.0004965675858373678


 Epoch: 7 Loss: 0.000490119987528845


 Epoch: 8 Loss: 0.0004848899140141465


 Epoch: 9 Loss: 0.0004799686655750852


# Читаємо між рядків

In [29]:
import math
import collections
from typing import List, Tuple


# Допоміжна фунція:
def get_letter(w1: str, w2: str) -> str:
    """Повертає літеру, якої відрізняються слова або пробіл для однакових слів.
    """

    letters1 = collections.Counter(w1)
    letters2 = collections.Counter(w2)

    diff = letters1 - letters2
    if len(diff) != 1:
        return " "

    return diff.most_common()[0][0]


def score_text(text: str, model, vocab) -> float:

    tokens = tokenize(text)

    total_log_prob = 0.0

    for context, target in prepare_data(tokens, model.context_len):

        X = vectorize(context, vocab)
        target = vectorize([target], vocab)[0]
        log_probs = model(X)
        target_log_prob = log_probs[0, target]
        total_log_prob += target_log_prob

    return torch.exp(torch.tensor(-total_log_prob / len(tokens))).item()


# Можете змінювати параметри та весь цей код, якщо потрібно
def solve(model, vocab, test_items) -> Tuple[List[str], str]:
    """Повертає список виправлених слів для кожного з текстів в test_items та
    секретне повідомлення.
    """

    choices = []
    secret = []

    for item in test_items:
        scores = []
        for corr in item['corrections']:

            # Підставляємо слово-кандидат в текст
            text = item['text'][:item['error_start']] + corr + item['text'][item['error_end']:]

            # Рахуємо score тексту
            score = score_text(text, model, vocab)
            scores.append(score)

            # print(f'{score:.4f} {text}')

        # Сортуємо кандидатів на виправлення за score
        result = sorted(zip(scores, item['corrections']), key=lambda x: x[0])

        # Обираємо найкращу заміну
        best = result[0]
        best_word = best[1]
        choices.append(best_word)

        # Знаходимо чергову літеру повідомлення
        error = item['error']
        letter = get_letter(error, best_word)
        secret.append(letter)

    secret_message = ''.join(secret)

    return choices, secret_message

choices, secret_message = solve(model, vocab, lab.test_items)

lab.evaluate_accuracy(choices)
print("SECRET MESSAGE: ", secret_message)


<ipython-input-29-3fb6fd3ce320>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.exp(torch.tensor(-total_log_prob / len(tokens))).item()


Відповідь правильна ✅
accuracy = 0.60. Вже краще. Можеш попрацювати над моделлю ще, а можеш рухатися далі й спробувати розгадати приховане повідомлення
SECRET MESSAGE:   к добре т  що смерті  е  о сьія і н  пи  ю ч   я ки  м й хр стсщоч  м боГо е   з кОанеп лонюся в п Редчутті  ед відомихнв рств щои и люб В   н    Б авс зскверн  н на ис   пРокль нувк  тт ин роде   й  о  ебе я щ  верну   В смезті оберн          тя св ї  стр жд нн мяі незл м об  ччям    си  то ілд зе  о пок    сь і чеС   г я улв чесні  во  Ві   і чесними с ь з     б ллюсь


In [19]:
lab.answer("Василь Стус")

Відповідь правильна ✅
Правильно! 🚀 Заповни тепер цю форму, будь ласка: https://tally.so/r/wkl0zZ


Відправте посилання на цей colab або PDF з ним на пошту oleksii.o.syvokon@lpnu.ua. Дякую!
